In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [3]:
tX.shape

(250000, 30)

## Cleaning data set

In [4]:
from helpers import *

def clean_data(tx):
    nbrRows = tx.shape[0]
    nbrColunms = tx.shape[1]
    tx_temp = np.zeros((nbrRows,nbrColunms))
    
    for columnID in range(nbrColunms):
        currentColumn = tx[:,columnID]

        nanIndices = np.where(currentColumn == -999)
        tempColumm = np.delete(currentColumn, nanIndices, axis=0)
        
        # replace -999 values with mean
        #mean = np.mean(tempColumm)
        #currentColumn[nanIndices] = mean

        # replace -999 values with median
        median = np.median(tempColumm)
        currentColumn[nanIndices] = median
        
        tx_temp[:,columnID] = currentColumn

    tx_cleaned, mean_, std_ = standardize(tx_temp)
        
    return tx_cleaned

In [5]:
tX_cleaned = clean_data(tX)

## Prediction

In [6]:
def split_data(y, x, seed=1):
    """split the dataset based on the split ratio."""
    # set seed
    np.random.seed(seed)
    
    # set mask
    ratio = 0.7
    msk = np.random.rand(len(y)) < ratio
    
    # training data set
    x_tr = x[msk]
    y_tr = y[msk]
    
    # test data set
    x_test = x[~msk]
    y_test = y[~msk]
    
    return x_tr, x_test, y_tr, y_test

In [7]:
def prediction(y, tx, gamma, lambda_, max_iters, method):
    # split data
    x_tr, x_test, y_tr, y_test = split_data(y, tx)
    
    # training
    loss = 0
    weights = []
    if method == 1:
        loss, weights = least_squares_GD(y_tr, x_tr, gamma, max_iters)
    elif method == 2:
        loss, weights = least_squares_SGD(y_tr, x_tr, gamma, max_iters)
    elif method == 3:
        loss, weights = least_squares(y_tr, x_tr)
    elif method == 4:
        loss, weights = ridge_regression(y_tr, x_tr, lambda_)
    elif method == 5:
        loss, weights = logistic_regression(y_tr, x_tr, gamma, max_iters)
    else:
        loss, weights = reg_logistic_regression(y_tr, x_tr, lambda_, gamma, max_iters)
        
    # compute prediction
    y_pred = predict_labels(weights, x_test)    
    
    # accuracy of the prediction
    N = y_test.shape[0]
    pred = np.sum(y_pred == y_test)/N
        
    return pred

# Implementation of ML methods

## Linear regression - gradient descent

In [8]:
from costs import *

def compute_gradient(y, tx, w):
    # error
    e = y - tx.dot(w)
    
    # gradient 
    N=y.shape[0]
    gradient = - np.transpose(tx).dot(e)/N
    
    return gradient

In [9]:
def least_squares_GD(y, tx, gamma, max_iters):
    # Define parameters to store w and loss
    w_init = np.zeros(tx.shape[1]) # initialization of the weight
    ws = [w_init]
    w_temp = w_init
    losses = []
    
    for n_iter in range(max_iters):
        # compute gradient and loss
        grad = compute_gradient(y, tx, w_temp)
        loss = compute_loss(y, tx, w_temp)
        
        # update w by gradient
        w_temp -= gamma*grad
        
        # store w and loss
        ws.append(np.copy(w_temp))
        losses.append(loss)

    return losses, ws[-1]

### Training

In [10]:
max_iters_GD = 100
gamma_GD = 1.0e-4
method = 1

loss_GD, weights_GD = least_squares_GD(y, tX_cleaned, gamma_GD, max_iters_GD)
pred_GD = prediction(y, tX_cleaned, gamma_GD, 0, max_iters_GD, method)

print("\nweights_GD:\n",weights_GD,"\n")
print("pred_GD = ", pred_GD)


weights_GD:
 [ -3.13111489e-03   2.47634036e-04  -3.29791393e-03  -1.29920711e-04
   1.77654650e-03   1.89639194e-03   1.97165872e-03  -1.70751341e-03
   1.41399486e-04  -1.60263082e-04   1.40353212e-03  -1.83498821e-03
   2.53646192e-03   1.63714600e-03   2.19609861e-03  -9.09083521e-06
  -4.16941769e-05  -3.05818690e-04   1.37716246e-05   3.89579620e-05
   1.88739443e-04   6.98964757e-05   1.23781974e-03   1.22455469e-03
   1.05283667e-03   6.66937883e-07   8.29890589e-06   1.91011203e-04
   5.53901786e-06  -3.28339782e-05   1.22353906e-03] 

pred_GD =  0.698416503523


## Linear regression - stochastic gradient descent

In [11]:
def compute_stoch_gradient(y, tx, w):
    B = 35 # size of the batch
    sum = 0
    for minibatch_y, minibatch_tx in batch_iter(y, tx, B):
        sum += compute_gradient(minibatch_y, minibatch_tx, w)

    return sum / B

In [12]:
def least_squares_SGD(y, tx, gamma, max_iters):    
    # Define parameters to store w and loss
    w_init = np.zeros(tx.shape[1]) # initialization of the weight
    ws = [w_init]
    w_temp = w_init
    losses = []

    for n_iter in range(max_iters):
        # compute gradient and loss
        grad = compute_stoch_gradient(y, tx, w_temp)
        loss = compute_loss(y, tx, w_temp)

        # update w by gradient
        w_temp -= gamma*grad
        
        # store w and loss
        ws.append(np.copy(w_temp))
        losses.append(loss)
        
    return losses, ws[-1]

### Training

In [13]:
max_iters_SGD = 100
gamma_SGD = 1.0e-4
method = 2

loss_SGD, weights_SGD = least_squares_SGD(y, tX_cleaned, gamma_SGD, max_iters_SGD)
print("\nweights_SGD:\n",weights_SGD,"\n")

pred_SGD = prediction(y, tX_cleaned, gamma_SGD, 0, max_iters_SGD, method)
print("pred_SGD = ", pred_SGD)


weights_SGD:
 [ -2.74638145e-01   1.60274052e-03  -1.95033071e-01  -3.37329259e-02
   5.45739079e-02   4.78355801e-02   5.10627558e-02  -2.67122450e-02
   9.53312660e-02  -3.39904324e-02   2.12236919e-02  -7.98159738e-02
   1.18977961e-01   6.47392730e-02   1.14147909e-01  -1.29029434e-03
  -2.55569329e-03   3.84243810e-02  -5.05299418e-04   2.57878683e-03
   2.50419205e-04   2.34648500e-03  -2.10050744e-03   3.35893033e-03
  -1.87064285e-03   1.23228641e-04   5.73741615e-04  -3.18989034e-02
   1.28898969e-03  -1.23166355e-03  -9.69651176e-03] 

pred_SGD =  0.729513764034


## Least squares

In [14]:
def least_squares(y, tx):
    # Compute optimum weight
    tx_transpose = np.transpose(tx)
    A = tx_transpose.dot(tx)
    b = tx_transpose.dot(y)
    w_opt = np.linalg.solve(A,b)
    
    # Compute loss
    loss = compute_loss(y, tx, w_opt)
        
    return loss, w_opt # returns loss, and optimal weights

### Training

In [15]:
method = 3

loss_LeastS, weights_LeastS = least_squares(y, tX_cleaned)
print("\nweights_LeastS:\n",weights_LeastS,"\n")

pred_LeastS = prediction(y, tX_cleaned, 0, 0, 0, method)
print("pred_LeastS = ", pred_LeastS)


weights_LeastS:
 [ -3.14664000e-01   1.22272262e-02  -2.53066586e-01  -2.63456197e-01
   1.29243443e-02   1.92805216e-02   1.04114629e-01   7.14391075e-03
   2.80223055e-01  -2.77613512e-02  -3.20624220e+02  -1.87675501e-01
   1.20153160e-01   7.45926202e-02   6.22887642e+01  -8.00284516e-04
  -8.11540969e-04   6.14298446e+01  -6.52146078e-04   2.55788779e-03
   1.00389486e-01   9.43326693e-04  -4.78438667e-02   5.46733536e-02
  -3.73249556e-02   5.84626551e-04   2.49121190e-04  -1.82119811e-02
   1.51617614e-03  -1.61520956e-03   2.71558175e+02] 

pred_LeastS =  0.746147801883


## Ridge regression

In [16]:
def ridge_regression(y, tx, lambda_):    
    # Initiation variables
    lamb_ = 2*len(y)*lambda_
    
    # Compute optimum weight
    tx_transpose = np.transpose(tx)
    A = np.dot(tx_transpose,tx) + lamb_*np.eye(tx.shape[1])
    b = tx_transpose.dot(y)
    w_opt = np.linalg.solve(A,b)
    
    # Compute loss
    loss = compute_loss(y, tx, w_opt)
    
    return loss, w_opt # returns mse, and optimal weights

### Training

In [28]:
lambda_RR = 2
method = 4

loss_RR, weights_RR = ridge_regression(y, tX_cleaned, lambda_RR)
print("\nweights_RR:\n",weights_RR,"\n")

pred_RR = prediction(y, tX_cleaned, 0, lambda_RR, 0, method)
print("pred_RR = ", pred_RR)


weights_RR:
 [ -6.29328000e-02   3.92207695e-03  -5.57484587e-02  -2.34782240e-03
   2.07312588e-02   2.51082306e-02   2.40525833e-02  -2.09190262e-02
   1.26030461e-02  -7.87099153e-03   1.27481195e-02  -2.95635675e-02
   3.83846442e-02   2.39936008e-02   3.35578838e-02  -2.55836514e-04
  -8.13934838e-04  -3.82291506e-03   5.71152754e-05   7.30465092e-04
  -2.76716941e-03   1.04514791e-03   9.55063769e-03   1.06616720e-02
   7.75571805e-03   1.28865676e-05   1.04631150e-04  -4.52908849e-03
   1.87095227e-04  -4.63448317e-04   8.23629627e-03] 

pred_RR =  0.71382529599


## Logistic regression

In [29]:
def sigmoid(t):
    return 1/(1+np.exp(-t))

In [30]:
def learning_by_gradient_descent(y, tx, w, gamma):
    # compute the loss
    N = tx.shape[0]
    l1 = tx.dot(w) + np.log(np.ones((N))+np.exp(-tx.dot(w)))
    l2 = y*(tx.dot(w))
    loss = (np.ones((1,N)).dot(l1-l2))[0]
    
    # compute the gradient
    grad = np.transpose(tx).dot(sigmoid(tx.dot(w))-y)
    
    # update w
    w = w - gamma*grad

    return loss, w

In [31]:
def logistic_regression(y, tx, gamma, max_iters):
    # init parameters
    threshold = 1e-8
    w_temp = np.zeros(tx.shape[1]) # initialization of the weight
    ws = [w_temp]
    losses = [8000]

    # start the logistic regression
    for iter in range(max_iters):        
        # get loss and update w.
        loss, w_temp = learning_by_gradient_descent(y, tx, w_temp, gamma)
        
        # store w and loss
        ws.append(np.copy(w_temp))
        losses.append(loss)
        
        # converge criteria
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    
    return losses, ws[-1]

### Training

In [34]:
max_iters_LogR = 1000
gamma_LogR = 1.0e-6
method = 5

loss_LogR, weights_LogR = logistic_regression(y, tX_cleaned, gamma_LogR, max_iters_LogR)
print("\nweights_LogR:\n",weights_LogR,"\n")

pred_LogR = prediction(y, tX_cleaned, gamma_LogR, 0, max_iters_LogR, method)
print("pred_LogR = ", pred_LogR)

/Users/jeannoel/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: RuntimeWarning: overflow encountered in exp
/Users/jeannoel/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: RuntimeWarning: overflow encountered in exp
  from ipykernel import kernelapp as app
/Users/jeannoel/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:18: RuntimeWarning: invalid value encountered in double_scalars



weights_LogR:
 [ -1.13508270e+02   6.50912498e-01  -5.22266414e+01  -3.07049847e+00
   1.41374308e+01   8.87152410e+00   9.74793611e+00  -6.88183593e+00
   1.85676153e+01  -5.09178200e+00   4.99574484e+00  -2.38045254e+01
   3.08866176e+01   1.09625279e+01   1.96436876e+01  -2.22938126e-01
  -6.11348279e-01  -3.12927753e+00   5.81765039e-02   3.12223291e-01
  -7.51870802e+00   5.69856646e-01   3.28048636e+00   4.10475885e+00
   2.00464918e+00   4.55253298e-03   1.15724245e-01  -4.66537008e+00
   1.48646888e-01  -2.13247224e-01   2.11017015e+00] 

pred_LogR =  0.714584415411


## Regularized logistic regression

In [ ]:
## Regularized logistic regression

### Training

In [ ]:
#max_iters_RLogR = 1000
#lambda_RLogR = 1.0e-2
#gamma_RLogR = 1.0e-8
#loss_RLogR, weights_RLogR = reg_logistic_regression(y, tX_cleaned, lambda_RLogR, gamma_RLogR, max_iters_RLogR)
#print("loss_RLogR={loss}\n".format(loss=loss_RLogR))
#print("weights_RLogR={w}".format(w=weights_RLogR[-1]))

### Cross-validation

In [16]:
def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold."""
    num_row = len(y)
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                 for k in range(k_fold)]
    
    return np.array(k_indices)


def cross_validation(y, x, k_indices, k, lambda_, degree):
    """return the loss of ridge regression."""
    # get k'th subgroup in test, others in train:
    x_test = x[k_indices[k]]
    y_test = y[k_indices[k]]
    
    tr_indices = np.delete(k_indices, k, axis=0)
    x_tr = np.delete(x, k, axis=0)
    y_tr = np.delete(y, k, axis=0)
    
    # form train and test data with polynomial basis function
    poly_x_tr = build_poly(x_tr, degree)
    poly_x_test = build_poly(x_test, degree)
    
    # calcualte weight and loss through least square.
    loss_tr, weight_tr = ridge_regression(y_tr, poly_x_tr, lambda_)
    loss_test, weight_te = ridge_regression(y_test, poly_x_test, lambda_)
    
    return loss_tr, loss_te, weight_tr, weight_te

### Bias-Variance Decomposition

# Generate predictions and save ouput in csv format for submission:

In [19]:
DATA_TEST_PATH = '../data/test.csv' 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
OUTPUT_PATH = '../data/dataSubmission_GR.csv' 
y_pred = predict_labels(weights_GD[-1], tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [ ]:
OUTPUT_PATH = '../data/dataSubmission_SGD.csv' 
y_pred = predict_labels(weights_SGD[-1], tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [ ]:
OUTPUT_PATH = '../data/dataSubmission_LS.csv' 
y_pred = predict_labels(weights_LeastS, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [43]:
OUTPUT_PATH = '../data/dataSubmission_RR.csv' 
y_pred = predict_labels(weights_RR, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [40]:
OUTPUT_PATH = '../data/dataSubmission_LogR.csv' 
y_pred = predict_labels(weights_LogR[-1], tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [ ]:
OUTPUT_PATH = '../data/dataSubmission_RLogR.csv' 
y_pred = predict_labels(weights_RLogR[-1], tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)